In [105]:
import statsmodels.api as sm
import datapackage
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from dateutil import parser
from datetime import datetime, timedelta

In [111]:
weather_data = pd.read_csv('weather_dataset.csv')

In [113]:
weather_data['Date'] = pd.to_datetime(weather_data['Date'], format='%d-%m-%y', errors='coerce')

In [114]:
print(weather_data)

           Date  prcp  tmin  tmax  tavg
0    2013-01-06   NaN  20.0  29.4  21.8
1    2013-02-06  13.2  20.0  27.6  22.3
2    2013-03-06   5.1  19.2  27.6  22.3
3    2013-04-06   4.6  19.7  29.9  24.1
4    2013-05-06   0.0  21.2  29.9  25.0
...         ...   ...   ...   ...   ...
3671 2023-11-26   0.0  16.9  30.3  23.6
3672 2023-11-27  35.1  16.9  31.3  24.1
3673 2023-11-28  10.9  16.9  27.1  22.0
3674 2023-11-29   1.6  18.5  24.9  21.7
3675 2023-11-30   0.0  19.3  27.9  23.6

[3676 rows x 5 columns]


In [115]:
weather_data['tmin'] = weather_data.groupby(weather_data['Date'].dt.to_period("M"))['tmin'].transform(lambda x: x.fillna(x.mean()))
weather_data['tmax'] = weather_data.groupby(weather_data['Date'].dt.to_period("M"))['tmax'].transform(lambda x: x.fillna(x.mean()))
weather_data['tavg'] = weather_data.groupby(weather_data['Date'].dt.to_period("M"))['tavg'].transform(lambda x: x.fillna(x.mean()))
print(weather_data)

           Date  prcp  tmin  tmax  tavg
0    2013-01-06   NaN  20.0  29.4  21.8
1    2013-02-06  13.2  20.0  27.6  22.3
2    2013-03-06   5.1  19.2  27.6  22.3
3    2013-04-06   4.6  19.7  29.9  24.1
4    2013-05-06   0.0  21.2  29.9  25.0
...         ...   ...   ...   ...   ...
3671 2023-11-26   0.0  16.9  30.3  23.6
3672 2023-11-27  35.1  16.9  31.3  24.1
3673 2023-11-28  10.9  16.9  27.1  22.0
3674 2023-11-29   1.6  18.5  24.9  21.7
3675 2023-11-30   0.0  19.3  27.9  23.6

[3676 rows x 5 columns]


In [117]:
weather_data['month'] = weather_data['Date'].dt.month

In [118]:
filtered_df = weather_data[weather_data['month'].isin([3, 4, 5, 6, 7, 8, 9, 10])]

In [119]:
monthly_means = filtered_df.groupby('month')['prcp'].mean()

In [120]:
for month in monthly_means.index:
    mean_value = monthly_means[month]
    weather_data.loc[weather_data['month'] == month, 'prcp'] = weather_data.loc[weather_data['month'] == month, 'prcp'].fillna(mean_value)

In [121]:
weather_data.drop('month', axis=1, inplace=True)

In [122]:
print(weather_data)

           Date  prcp  tmin  tmax  tavg
0    2013-01-06   NaN  20.0  29.4  21.8
1    2013-02-06  13.2  20.0  27.6  22.3
2    2013-03-06   5.1  19.2  27.6  22.3
3    2013-04-06   4.6  19.7  29.9  24.1
4    2013-05-06   0.0  21.2  29.9  25.0
...         ...   ...   ...   ...   ...
3671 2023-11-26   0.0  16.9  30.3  23.6
3672 2023-11-27  35.1  16.9  31.3  24.1
3673 2023-11-28  10.9  16.9  27.1  22.0
3674 2023-11-29   1.6  18.5  24.9  21.7
3675 2023-11-30   0.0  19.3  27.9  23.6

[3676 rows x 5 columns]


In [151]:
weather_data.to_csv("weather.csv", index=False)

In [124]:
weather_data['month'] = weather_data['Date'].dt.month

In [127]:
mode_df = weather_data[weather_data['month'].isin([11, 12, 1, 2])]

In [128]:
def fill_mode(group):
    mode_value = group.mode().iloc[0]
    return group.fillna(mode_value)

In [131]:
mode_df['prcp'] = mode_df.groupby('month')['prcp'].transform(fill_mode)

<ipython-input-131-8416673340b8>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mode_df['prcp'] = mode_df.groupby('month')['prcp'].transform(fill_mode)


In [147]:
weather_data = pd.merge(weather_data, mode_df[['Date', 'prcp']], on='Date', how='left')

In [148]:
weather_data['prcp'].fillna(filtered_df['prcp'], inplace=True)

In [149]:
weather_data.drop('month', axis=1, inplace=True)

In [150]:
print(weather_data)

           Date  prcp_x  tmin  tmax  tavg  prcp_y  prcp
0    2013-01-06     NaN  20.0  29.4  21.8     0.0   0.0
1    2013-02-06    13.2  20.0  27.6  22.3    13.2  13.2
2    2013-03-06     5.1  19.2  27.6  22.3     NaN   NaN
3    2013-04-06     4.6  19.7  29.9  24.1     NaN   NaN
4    2013-05-06     0.0  21.2  29.9  25.0     NaN   NaN
...         ...     ...   ...   ...   ...     ...   ...
3671 2023-11-26     0.0  16.9  30.3  23.6     0.0   0.0
3672 2023-11-27    35.1  16.9  31.3  24.1    35.1  35.1
3673 2023-11-28    10.9  16.9  27.1  22.0    10.9  10.9
3674 2023-11-29     1.6  18.5  24.9  21.7     1.6   1.6
3675 2023-11-30     0.0  19.3  27.9  23.6     0.0   0.0

[3676 rows x 7 columns]


In [152]:
# Combine prcp, prcp_x, and prcp_y into one column called prcp
weather_data['prcp'] = weather_data['prcp'].combine_first(weather_data['prcp_x']).combine_first(weather_data['prcp_y'])

# Drop columns prcp_x and prcp_y if they are no longer needed
weather_data.drop(['prcp_x', 'prcp_y'], axis=1, inplace=True)

In [153]:
print(weather_data)

           Date  tmin  tmax  tavg  prcp
0    2013-01-06  20.0  29.4  21.8   0.0
1    2013-02-06  20.0  27.6  22.3  13.2
2    2013-03-06  19.2  27.6  22.3   5.1
3    2013-04-06  19.7  29.9  24.1   4.6
4    2013-05-06  21.2  29.9  25.0   0.0
...         ...   ...   ...   ...   ...
3671 2023-11-26  16.9  30.3  23.6   0.0
3672 2023-11-27  16.9  31.3  24.1  35.1
3673 2023-11-28  16.9  27.1  22.0  10.9
3674 2023-11-29  18.5  24.9  21.7   1.6
3675 2023-11-30  19.3  27.9  23.6   0.0

[3676 rows x 5 columns]


In [154]:
weather_data.to_csv("weather_preprocess.csv", index=False)